### ***Libraries import***

In [24]:
import os
import pandas as pd
import numpy as np

### ***Data loading***

In [25]:
dataset_csv_path = '/kaggle/input/improved-cicids2017-and-csecicids2018/CICIDS2017_improved'

csv_file_names = ['monday.csv', 'thursday.csv', 'wednesday.csv', 'tuesday.csv','friday.csv']

complete_paths = []

for csv_file_name in csv_file_names:
    complete_paths.append(os.path.join(dataset_csv_path, csv_file_name))

df = pd.concat(map(pd.read_csv, complete_paths), ignore_index=True)

In [26]:
#dataset features
dict(df.dtypes)

{'id': dtype('int64'),
 'Flow ID': dtype('O'),
 'Src IP': dtype('O'),
 'Src Port': dtype('int64'),
 'Dst IP': dtype('O'),
 'Dst Port': dtype('int64'),
 'Protocol': dtype('int64'),
 'Timestamp': dtype('O'),
 'Flow Duration': dtype('int64'),
 'Total Fwd Packet': dtype('int64'),
 'Total Bwd packets': dtype('int64'),
 'Total Length of Fwd Packet': dtype('int64'),
 'Total Length of Bwd Packet': dtype('int64'),
 'Fwd Packet Length Max': dtype('int64'),
 'Fwd Packet Length Min': dtype('int64'),
 'Fwd Packet Length Mean': dtype('float64'),
 'Fwd Packet Length Std': dtype('float64'),
 'Bwd Packet Length Max': dtype('int64'),
 'Bwd Packet Length Min': dtype('int64'),
 'Bwd Packet Length Mean': dtype('float64'),
 'Bwd Packet Length Std': dtype('float64'),
 'Flow Bytes/s': dtype('float64'),
 'Flow Packets/s': dtype('float64'),
 'Flow IAT Mean': dtype('float64'),
 'Flow IAT Std': dtype('float64'),
 'Flow IAT Max': dtype('int64'),
 'Flow IAT Min': dtype('int64'),
 'Fwd IAT Total': dtype('int64'),
 '

In [27]:
#dataset shape
df.shape

(2099976, 91)

### ***Data preprocessing***

**Manage Categorical features**

In [28]:
cat_columns = ['Flow ID', 'Src IP', 'Dst IP', 'Timestamp']
df.drop(cat_columns, axis=1, inplace=True)
print('Num of features after deleting categorical variables: ', len(df.columns))

Num of features after deleting categorical variables:  87


**Attempted Category Relabelling + unnecessary features removal**

In [29]:
df.loc[df['Attempted Category'] != -1, 'Label'] = 'BENIGN'

In [30]:
df.drop(['id', 'Attempted Category', 'Src Port', 'Dst Port', 'Protocol'], axis=1, inplace=True)
print('Num of features after deleting uncorrelated variables: ', len(df.columns))

Num of features after deleting uncorrelated variables:  82


**Manage missing and duplicated values**

In [31]:
#check missing values
def check_missing_values(df):
    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    missing_values_count = df.isnull().sum()
    missing_values_count = missing_values_count[missing_values_count > 0].sort_values(ascending=False)
    print(missing_values_count)

check_missing_values(df)

Flow Bytes/s      5
Flow Packets/s    5
dtype: int64


In [32]:
#drop null values
df.dropna(inplace=True)
df.shape

(2099971, 82)

In [33]:
#drop duplicates
df = df.drop_duplicates()
df.shape

(1715326, 82)

In [34]:
#drop columns with identical value in all records
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
df.shape

(1715326, 82)

**Relabel all records as Benign (0) or Anomalous(1)**

In [35]:
df.loc[df['Label'] == 'BENIGN', 'Label'] = 0
df.loc[df['Label'] != 0, 'Label'] = 1

df['Label'].value_counts()

Label
0    1432918
1     282408
Name: count, dtype: int64

**Remove high correlated features**

In [36]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, t=0.9):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[au_corr > t]

high_corr = get_top_abs_correlations(df)

In [37]:
drop_cols = []
for item in high_corr.items():
    drop_cols.append(item[0][0])

drop_cols

['Fwd Packet Length Mean',
 'Packet Length Mean',
 'Bwd Packet Length Mean',
 'Total Bwd packets',
 'Total Fwd Packet',
 'Fwd URG Flags',
 'Flow Duration',
 'Total Fwd Packet',
 'Flow IAT Max',
 'Flow IAT Max',
 'Total Fwd Packet',
 'Fwd IAT Max',
 'Total Length of Bwd Packet',
 'Total Bwd packets',
 'Idle Mean',
 'Bwd Packet Length Max',
 'Packet Length Max',
 'Average Packet Size',
 'Packet Length Mean',
 'Idle Mean',
 'Flow IAT Max',
 'Fwd IAT Max',
 'Bwd Packet Length Max',
 'Bwd Packet Length Max',
 'Flow Duration',
 'Fwd IAT Total',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'Bwd Packet Length Mean',
 'Bwd Bytes/Bulk Avg',
 'Fwd Segment Size Avg',
 'Fwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Fwd IAT Mean',
 'Average Packet Size',
 'Packet Length Mean',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Fwd RST Flags',
 'Bwd Packet Length Mean',
 'Packet Length Std',
 'Flow IAT Max',
 'Bwd IAT Max',
 'Fwd IAT Max',
 'Bwd PSH Flags'

In [38]:
df = df.drop(columns=drop_cols, axis=1)
df.shape

(1715326, 49)

**Save preprocessed dataset as csv**

In [ ]:
df.to_csv('/kaggle/output/CICIDS2017_improved-preprocessed.csv')